In [ ]:
"""
このコードは、学習率をより大きく調整し、最大反復回数を増やしてより多くの学習を行うようにしました。
これらの変更は、フォルド1、4、5での誤差を減少させるのに役立つかもしれません。
"""
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

# データファイルのパス設定
file_path = 'click.csv'

try:
    # 学習データの読み込み
    train = np.loadtxt(file_path, delimiter=',', dtype='int', skiprows=1)
    train_x = train[:, 0]
    train_y = train[:, 1]
except Exception as e:
    print(f"データの読み込みエラー: {file_path}からのデータの読み込みに失敗しました: {e}")
    exit(1)

# パラメータの初期化
theta0 = np.random.rand()
theta1 = np.random.rand()

# 予測関数の定義
def f(x):
    return theta0 + theta1 * x

# 目的関数の定義（誤差の計算）
def E(x, y):
    return 0.5 * np.sum((y - f(x)) ** 2)

# 標準化関数の定義
mu = train_x.mean()
sigma = train_x.std()

def standardize(x):
    return (x - mu) / sigma

train_z = standardize(train_x)

# 学習率と初期設定
ETA = 5e-3  # 学習率を大きく調整
max_iterations = 10000  # 最大反復回数を増やす
diff = 1
count = 0
error = E(train_z, train_y)

# k-foldクロスバリデーションの設定
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_idx = 1

# クロスバリデーションの反復
for train_idx, val_idx in kfold.split(train_z):
    print(f"Fold {fold_idx}:")

    # トレーニングセットとバリデーションセットの準備
    train_z_fold, val_z_fold = train_z[train_idx], train_z[val_idx]
    train_y_fold, val_y_fold = train_y[train_idx], train_y[val_idx]

    # 学習反復（最大反復回数までまたは誤差が0.01未満になるまで）
    while diff > 1e-2 and count < max_iterations:
        try:
            # パラメータの更新
            tmp_theta0 = theta0 - ETA * np.sum((f(train_z_fold) - train_y_fold))
            tmp_theta1 = theta1 - ETA * np.sum((f(train_z_fold) - train_y_fold) * train_z_fold)

            # パラメータ値の更新
            theta0, theta1 = tmp_theta0, tmp_theta1

            # 新しい誤差の計算
            current_error = E(train_z_fold, train_y_fold)
            diff = error - current_error
            error = current_error

            # 学習ログの出力
            count += 1
            log = '{}回目: theta0 = {:.3f}, theta1 = {:.3f}, 差分 = {:.4f}'
            print(log.format(count, theta0, theta1, diff))
        except OverflowError as e:
            print(f"オーバーフローエラー: {e}")
            break
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            break

    # バリデーションセットの性能評価
    val_error = E(val_z_fold, val_y_fold)
    print(f"フォルド {fold_idx} の検証エラー: {val_error}\n")

    # フォルドインデックスの増加
    fold_idx += 1

# 最終的な学習結果の可視化
try:
    x = np.linspace(-3, 3, 100)
    plt.plot(train_z, train_y, 'o')
    plt.plot(x, f(x))
    plt.xlabel('標準化された x')
    plt.ylabel('y')
    plt.title('線形回帰のフィット')
    plt.show()
except Exception as e:
    print(f"プロット中にエラーが発生しました: {e}")
